In [ ]:
# p.class=`znach`
uk brown


In [225]:
import tokenize_uk
import pymorphy2
import requests
import collections
from lxml import html
import itertools
import re
from sklearn.metrics import f1_score, accuracy_score

In [26]:
%run -n ukr_stemmer3.py

In [182]:
word = 'земля'

In [183]:
conversion = {
    u'\u0410' : 'A',    u'\u0430' : 'a',
    u'\u0411' : 'B',    u'\u0431' : 'b',
    u'\u0412' : 'V',    u'\u0432' : 'v',
    u'\u0413' : 'Gh',    u'\u0433' : 'gh',
    u'\u0490' : 'G',    u'\u0491' : 'g',
    u'\u0414' : 'D',    u'\u0434' : 'd',
    u'\u0415' : 'E',    u'\u0435' : 'e',
    u'\u0404' : 'Ye',   u'\u0454' : 'je',
    u'\u0416' : 'Zh',   u'\u0436' : 'zh',
    u'\u0417' : 'Z',    u'\u0437' : 'z',
    u'\u0418' : 'Y',    u'\u0438' : 'y',
    u'\u0406' : 'I',    u'\u0456' : 'i',
    u'\u0407' : 'Ji',   u'\u0457' : 'ji',
    u'\u0419' : 'J',    u'\u0439' : 'j',
    u'\u041a' : 'K',    u'\u043a' : 'k',
    u'\u041b' : 'L',    u'\u043b' : 'l',
    u'\u041c' : 'M',    u'\u043c' : 'm',
    u'\u041d' : 'N',    u'\u043d' : 'n',
    u'\u041e' : 'O',    u'\u043e' : 'o',
    u'\u041f' : 'P',    u'\u043f' : 'p',
    u'\u0420' : 'R',    u'\u0440' : 'r',
    u'\u0421' : 'S',    u'\u0441' : 's',
    u'\u0422' : 'T',    u'\u0442' : 't',
    u'\u0423' : 'U',    u'\u0443' : 'u',
    u'\u0424' : 'F',    u'\u0444' : 'f',
    u'\u0425' : 'Kh',    u'\u0445' : 'kh',
    u'\u0426' : 'C',   u'\u0446' : 'c',
    u'\u0427' : 'Ch',   u'\u0447' : 'ch',
    u'\u0428' : 'Sh',   u'\u0448' : 'sh',
    u'\u0429' : 'Shh',  u'\u0449' : 'shh',
    u'\u044c' : 'j',
    u'\u042e' : 'Ju',   u'\u044e' : 'ju',
    u'\u042f' : 'Ja',   u'\u044f' : 'ja',
    u'\'' : '.'
}

def transliterate(word):
    translit = []
    for c in word:
        translit.append(conversion.get(c, c))
    return ''.join(translit)

In [184]:
URL = "http://sum.in.ua/s/"

def get_senses(word):
    """
    Extract senses for WORD from the dictionary.
    """
    word = transliterate(word)
    page = requests.get(URL + word)
    tree = html.fromstring(page.text)
    senses = tree.xpath("//p[@class='znach']")
    print(len(senses), "senses found.")
    return list(enumerate([" ".join(i.xpath('text()')) for i in senses], 1))

In [185]:
get_senses(word)

6 senses found.


[(1,
  '   Третя по порядку від\nСонця велика планета, яка обертається навколо своєї\nосі і навколо Сонця. І сонечко серед неба Опинилось-стало,\n Мов жених той молодую, Землю оглядало  ; Хай дітям нашим на потребу Ростуть\nмогутні кораблі, Нехай розвідниками в небо Летять\nсупутники Землі!  ;  \xa0 Місце життя і діяльності людей. [Неофіт-раб:]\nПаненяті аж два щастя буде, одно тут, на землі,\nа друге в небі...  ; — Я не знаю,\nчи там угорі, наді мною, є життя, але я добре бачу, що\nвся краса тут, на землі, — роздумував він  ; \xa0  Далеко, як небо від землі  .\n    див.\n .\n'),
 (2,
  "   Верхній шар земної кори. Яка була\nмерзла земля, зараз уся порозтавала  ; На всьому, на всьому борні їх печать, в воді й під\nземлею їх кості лежать  ;  \xa0  \nЗемляна поверхня, площина, по якій ходять. Брязнуло\nдо землі намисто й розкотилось по куточках  ; Дерева посхилялись під вагою плодів,\nі плоди падали на землю й лежали на ній  .\n    див.  ;\n  див.   1;\n ;   —\nзавмерти, застигнути на мі

In [186]:
def word_sign(word):
    senses = get_senses(word)
    signs = []
    for i,sense in senses:
        tokens = tokenize_uk.tokenize_words(sense)
        lower = map(lambda t: t.lower(), tokens)
        stems = map(lambda t: UkrainianStemmer(t).stem_word(), lower)
        signs.append((i, set(stems)))
    return signs

In [187]:
" ".join(word_sign(word)[1][1])

6 senses found.


'несподіван все кор до велик вол а зіщулил мат полюб серц бач козак вниз домовин раптов праворуч семер дівчат переживан несподіванк отаман інокент мов пол нас заповзятлив жит з бідов став шан та благородств остав удар об земн ким всьом низьк порозтавал за гожіст храпк ні , жінк про провал де топч земл їм печ як ще кинут у є застигнут посхилял усміхнет замовк зникнут мож ... ліворуч своє дістан ваг і — дерев бород вросл бул труднощ три завмерт дуж він . безслідн бут хорош тут пер те лиц 1 від настирлив вмерт зметут роздобув чуж плод ход вод зараз розкотил знайшл борн ог дос померл дів сильн виявляюч внутрішн був біл б вираз закид - так понур лих неха мотор люданськ т марі чужин почутт заруб ! ) нам куточк дев виринул згад : розум враз ляж на блиснул ( особлив намист перед чапл дістав ін нагот пар вирв жив ж колгоспник мен площин скид то нем наш видер я дивит уродил ког верхн поверхн кост по жахну ? пок кон місц хлопц гарм голов леж знищув ї хіб мо й полов під незручност пішл ] дощент па

In [188]:
def prep_sent(sent):
    tokens = tokenize_uk.tokenize_words(sent)
    lower = map(lambda t: t.lower(), tokens)
    stems = set(map(lambda t: UkrainianStemmer(t).stem_word(), lower))
    return stems
    

In [189]:
import glob

BROWN = '/Users/serhiinechyporhuk/courses/corpus/data/good'

def sent_with_words(word):
    sentences = []
    for path in glob.glob(BROWN + '/*'):
        with open(path) as f:
            text = f.read()
            t = itertools.dropwhile(lambda x: x != '<body>',  text.split('\n'))
            t = list(t)[1:-1]
            t = " ".join(t)
            t = re.sub(r'\</?\w+\>', '', t)
            sents = tokenize_uk.tokenize_sents(t)
            for sent in sents:
                stems = prep_sent(sent)
                if UkrainianStemmer(word.lower()).stem_word() in stems:
                    sentences.append(sent)
    return sentences

In [191]:
sents_with_word = sent_with_words(word)
print(len(sents_with_word))

196


In [199]:
def lesk(signs, sent, verbose=False):
    if verbose:
        print(sent)
    best_sense = 0
    best_score = 0
    for i, sign in signs:
        common = set(sign) & set(prep_sent(sent))
        if verbose:
            print("Common words({}): {}".format(len(common), common))
        score = len(common)
        if score > best_score:
            best_score = score
            best_sense = i-1
    return signs[best_sense][0]

In [200]:
lesk(word_sign(word), sents_with_word[15])

6 senses found.


2

In [201]:
with open('test1.txt', 'tw') as f:
    sents = sents_with_word[10:60]
    senses = word_sign(word)
    for sent in sents:
        f.write("{}\t{}\n".format(lesk(senses, sent), sent))

6 senses found.


In [206]:
def load_test():
    with open('test.txt') as f:
        return [tuple(line.split('\t')) for line in f.readlines()]

In [285]:
def evaluate(predictor, word):
    signs = word_sign(word)
    test = load_test()
    preds = []
    for true, sent in test:
        pred = predictor(signs, sent)
        preds.append(pred)
    print('Acc:', accuracy_score([int(t[0]) for t in test], preds))

In [286]:
evaluate(lesk, word)

6 senses found.
Acc: 0.391304347826087


In [287]:
import numpy as np

def idf_lesk(signs, sent, verbose=False):
    if verbose:
        print(sent)
    best_sense = 0
    best_score = -1e12
    all_weights = []
    for i, sign in signs:
        ord_sign = list(sign)
        weights = {}
        for token in ord_sign:
            his_signs = sum([1 if token in sign2 else 0 for _, sign2 in signs])
            token_score = -np.log(1/his_signs)
            weights[token] = token_score
        all_weights.append(weights)
            
    for i, sign in signs:
        weights = all_weights[i-1]
        common = set(sign) & set(prep_sent(sent))
        if verbose:
            print("Common words({}): {}".format(len(common), [(c, round(weights[c],2)) for c in common]))
        matched = [weights.get(token, 0) for token in prep_sent(sent)]
        score = sum(matched)
        if score > best_score:
            best_score = score
            best_sense = i-1
    print(best_score, best_sense+1)
    return signs[best_sense][0]

In [288]:
idf_lesk(word_sign(word), sents_with_word[4], verbose=True)

6 senses found.
Та, мабуть, то був чи не єдиний президент в історії людства, котрий того президентства не жадав, натомість просто мріяв бути собою на своїй землі.
Common words(8): [('на', 1.61), ('то', 1.39), (',', 1.79), ('в', 1.79), ('.', 1.79), ('чи', 0.69), ('земл', 1.79), ('не', 1.39)]
Common words(10): [('на', 1.61), ('то', 1.39), (',', 1.79), ('в', 1.79), ('.', 1.79), ('був', 0.69), ('бут', -0.0), ('та', 1.1), ('земл', 1.79), ('не', 1.39)]
Common words(10): [('на', 1.61), ('то', 1.39), (',', 1.79), ('в', 1.79), ('.', 1.79), ('чи', 0.69), ('був', 0.69), ('та', 1.1), ('земл', 1.79), ('не', 1.39)]
Common words(5): [('на', 1.61), (',', 1.79), ('в', 1.79), ('.', 1.79), ('земл', 1.79)]
Common words(7): [('на', 1.61), (',', 1.79), ('в', 1.79), ('.', 1.79), ('та', 1.1), ('земл', 1.79), ('не', 1.39)]
Common words(6): [('то', 1.39), ('в', 1.79), (',', 1.79), ('соб', -0.0), ('.', 1.79), ('земл', 1.79)]
14.033971161374101 3


3

In [290]:
evaluate(lambda signs, word: idf_lesk(signs, word, True), word)

6 senses found.
Про безліч сніжинок, схожих на снігові квіти, що танцюють свій веселий танець, спускаючись на землю.

Common words(5): [('на', 1.61), (',', 1.79), ('.', 1.79), ('земл', 1.79), ('що', 1.1)]
Common words(6): [('на', 1.61), (',', 1.79), ('про', 0.69), ('.', 1.79), ('земл', 1.79), ('що', 1.1)]
Common words(5): [('на', 1.61), (',', 1.79), ('.', 1.79), ('земл', 1.79), ('що', 1.1)]
Common words(4): [(',', 1.79), ('.', 1.79), ('на', 1.61), ('земл', 1.79)]
Common words(6): [('на', 1.61), (',', 1.79), ('про', 0.69), ('.', 1.79), ('земл', 1.79), ('весел', -0.0)]
Common words(3): [(',', 1.79), ('.', 1.79), ('земл', 1.79)]
8.77647578934632 2
Бо наступні сніжинки, лапаті й пухнасті, вже повільно опускалися на землю.

Common words(4): [(',', 1.79), ('.', 1.79), ('на', 1.61), ('земл', 1.79)]
Common words(6): [('на', 1.61), (',', 1.79), ('.', 1.79), ('вже', 0.69), ('земл', 1.79), ('й', 1.61)]
Common words(5): [('на', 1.61), (',', 1.79), ('.', 1.79), ('земл', 1.79), ('й', 1.61)]
Common w

Common words(11): [('на', 1.61), (',', 1.79), ('про', 0.69), ('а', 1.39), ('.', 1.79), ('!', 1.61), ('земл', 1.79), ('й', 1.61), ('як', 1.61), ('не', 1.39), ('одн', 0.69)]
Common words(11): [('хоч', -0.0), ('то', 1.39), (',', 1.79), ('.', 1.79), ('!', 1.61), ('де', 0.69), ('земл', 1.79), ('-', 1.39), ('й', 1.61), ('рідн', -0.0), ('є', 1.1)]
24.69514441560774 2
Але в цей раз Аліна відчуває, що крил немає, а вона з шаленою швидкістю летить назустріч землі і… прокидається, не знаючи, чи вижила вона там, у сні.

Common words(11): [('в', 1.79), ('там', 0.69), (',', 1.79), ('.', 1.79), ('а', 1.39), ('чи', 0.69), ('земл', 1.79), ('і', 1.79), ('що', 1.1), ('ал', -0.0), ('не', 1.39)]
Common words(12): [('нем', -0.0), ('в', 1.79), (',', 1.79), ('.', 1.79), ('а', 1.39), ('і', 1.79), ('з', 0.69), ('земл', 1.79), ('раз', -0.0), ('що', 1.1), ('у', 0.69), ('не', 1.39)]
Common words(12): [('це', 0.69), ('в', 1.79), ('там', 0.69), (',', 1.79), ('.', 1.79), ('а', 1.39), ('чи', 0.69), ('з', 0.69), ('земл

Common words(9): [(',', 1.79), ('мов', 0.69), ('в', 1.79), ('.', 1.79), ('а', 1.39), ('земл', 1.79), ('—', 1.79), ('і', 1.79), ('що', 1.1)]
Common words(11): [('мов', 0.69), (',', 1.79), ('в', 1.79), ('.', 1.79), ('а', 1.39), ('і', 1.79), ('з', 0.69), ('земл', 1.79), ('—', 1.79), ('й', 1.61), ('що', 1.1)]
Common words(12): [('це', 0.69), ('йог', -0.0), (',', 1.79), ('в', 1.79), ('.', 1.79), ('а', 1.39), ('і', 1.79), ('з', 0.69), ('земл', 1.79), ('—', 1.79), ('й', 1.61), ('що', 1.1)]
Common words(7): [(',', 1.79), ('в', 1.79), ('.', 1.79), ('і', 1.79), ('земл', 1.79), ('—', 1.79), ('й', 1.61)]
Common words(9): [('це', 0.69), ('в', 1.79), (',', 1.79), ('.', 1.79), ('а', 1.39), ('і', 1.79), ('земл', 1.79), ('—', 1.79), ('й', 1.61)]
Common words(8): [(',', 1.79), ('в', 1.79), ('.', 1.79), ('і', 1.79), ('земл', 1.79), ('—', 1.79), ('й', 1.61), ('рідн', -0.0)]
16.23119573871032 2
Втекти? Але мої ноги стали бамбуком, котрий вмить пустив глибокі паростки в землю, а язик розпух і прилип до горт